# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/anshul/spark-3.0.0-preview2-bin-hadoop2.7/')

## spark session

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('clustering').getOrCreate()

In [3]:
data=spark.read.csv('hack_data.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [6]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

### assemble the data

In [7]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler=VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],
outputCol='features')

In [9]:
new_data=assembler.transform(data)

### scale the model

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler=StandardScaler(inputCol='features',outputCol='scaled_features')

In [13]:
final_data=scaler.fit(new_data).transform(new_data)

In [16]:
final_data.select(['scaled_features']).show()

+--------------------+
|     scaled_features|
+--------------------+
|[0.56785108466505...|
|[1.41962771166263...|
|[2.20042295307707...|
|[0.14196277116626...|
|[1.41962771166263...|
|[0.07098138558313...|
|[1.27766494049636...|
|[1.56159048282889...|
|[1.06472078374697...|
|[0.85177662699757...|
|[1.06472078374697...|
|[2.27140433866020...|
|[1.63257186841202...|
|[0.63883247024818...|
|[1.91649741074455...|
|[0.85177662699757...|
|[1.49060909724576...|
|[0.70981385583131...|
|[1.41962771166263...|
|[1.56159048282889...|
+--------------------+
only showing top 20 rows



### prepare the clusters

In [17]:
from pyspark.ml.clustering import KMeans

### have to identify between 2 or 3 hackers

In [40]:
kmeans2=KMeans(featuresCol='scaled_features',k=2)
kmeans3=KMeans(featuresCol='scaled_features',k=3)

In [58]:
pred_2=kmeans2.fit(final_data).transform(final_data)
pred_3=kmeans3.fit(final_data).transform(final_data)

In [59]:
from pyspark.ml.evaluation import ClusteringEvaluator
eval=ClusteringEvaluator()

### evaluating both the clusters

In [60]:
result_kmeans2=eval.evaluate(pred_2)
result_kmeans3=eval.evaluate(pred_3)

In [61]:
print('cluster k=2 : ',result_kmeans2)
print('cluster k=3 : ',result_kmeans3)

cluster k=2 :  0.6683623593283755
cluster k=3 :  0.30412315937808737


we already know of the k value increases then the score will decrease. We can atleast say that having 3 clusters include more different elements in the same cluster. But still the result is not correctly found.

The last hint says that the attacks must be evenly divided between the hackers, we can simply count the number of entries in each prediction type for both cluster.

In [62]:
pred_3.select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   88|
|         0|   79|
+----------+-----+



### it doesnot have evenly divided attacks 

In [63]:
pred_2.select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



### so the attacks are evenly divided between two hackers, so it confirms that there were only two attackers!

## Bingo!